In [10]:
import astropy.units as u
import astropy.constants as acon
import numpy as np
import sympy as sp
from sympy.solvers.solveset import linsolve
from collections import defaultdict
si = {0 : u.kg , 1:  u.m , 2: u.s, 3 :u.Kelvin, 4 : u.A,5 : u.mol, 6: u.cd, 7: u.rad}

def getdim(q):
    q = q*1
    q = q.si.decompose()
    sb = q.unit.bases
    sp = q.unit.powers
    dictdim = defaultdict(lambda : 0)
    for i in range(len(sb)):
        dictdim[sb[i]] = sp[i]
    return dictdim

In [11]:
constants = [acon.c, acon.hbar, acon.k_B, acon.eps0]
units = [u.eV,u.mol,u.cd,u.rad]
targetunits = constants + units

In [3]:
constants = [4*np.pi*acon.eps0]
units = [u.kg,u.m,u.s,u.K,u.mol,u.cd,u.rad]
targetunits = constants + units

In [12]:
# This generates a dictionary that has the dimensions of the constants + units of the new unit system
rhsdict = defaultdict(lambda: 0)
for i in range(len(targetunits)):
    rhsdict[i] = getdim(targetunits[i])
    
def findtotal(t) :
    total = 0
    for i in range(len(targetunits)) :
        total = total + eval('r'+str(i+1))* rhsdict[i][t]
    return total

# Generating the variables r1, r2, r3... and e1,e2,e3... which are the exponents of the constants on RHS
for i in range(len(targetunits)) :
    globals()['r'+str(i+1)] = sp.symbols('r'+str(i+1))
    
for j in range(len(si)) :
    globals()['e'+str(j+1)] = findtotal(si[j])

# Generating the variables l1, l2, l3... which are the exponents of the constants on LHS
for i in range(len(targetunits)) :
    globals()['l'+str(i+1)] = sp.symbols('l'+str(i+1))

In [13]:
equations = [e1 - l1 , e2 - l2 , e3 - l3 , e4 - l4 , e5 - l5 , e6 - l6 , e7 - l7 , e8 - l8]

In [14]:
sol = linsolve(equations,(r1,r2,r3,r4,r5,r6,r7,r8))
solns = eval(str(sol.args[0]))
solncons = solns[0:(len(constants))]
solnunits = solns[len(constants) :] 

In [15]:
def convfactor(q):
    dimdict = getdim(q)
    a1 = dimdict[u.kg]
    a2 = dimdict[u.m]
    a3 = dimdict[u.s]
    a4 = dimdict[u.K]
    a5 = dimdict[u.A]
    a6 = dimdict[u.mol]
    a7 = dimdict[u.cd]
    a8 = dimdict[u.rad]
    factor = 1
    returnunits = 1
    for i in range(len(constants)):
        factor = factor * constants[i]**float(solncons[i].subs(l1,a1).subs(l2,a2).subs(l3,a3).subs(l4,a4).subs(l5,a5).subs(l6,a6).subs(l7,a7).subs(l8,a8))
    for i in range(len(units)):
        returnunits = returnunits * units[i]**float(solnunits[i].subs(l1,a1).subs(l2,a2).subs(l3,a3).subs(l4,a4).subs(l5,a5).subs(l6,a6).subs(l7,a7).subs(l8,a8))
    return factor, returnunits

In [16]:
def convert(q):
    f,r = convfactor(q)
    return (q/f).to(r)

In [18]:
convert(u.C)

<Quantity 1.89006702e+18>